In [8]:
from wealth_process_V_opt import *
from optim import *
from decimal import *
import pandas as pd
import warnings
import time

# Expected discounted dividend Maximization

In [9]:
lam ,b = 6, 3.125 # block discovery rate of the network and reward for finding a block
# Rate of discovery by the network is 6 blocks per hour and the block finding reward is 3.125
c = lam * b / 1.3  # operational cost per time unit (profitability on average)
X_solo = wealth_process(lam, b, c, 0, 1) # Define the wealth process for a solo miner
beta = 0.5  # Define the ruin probability threshold
x = sc.optimize.root_scalar(lambda x: X_solo.ruin_proba(x) - beta, bracket=[0.01, 1000], method='bisect').root  # initial wealth of the miner
print(x)
# Compute the expected value of the wealth process at time T=1
X_solo.E(x), X_solo.Var()
q = 0.5 # Define the discount rate
X_solo.V(x,q)[1]

3.9393322762496124


6.008253623015771

### Best of the pools

In [10]:
f0, f1, f2, f3 =0, 0.005, 0.01, 0.1  # pool fees
delta0, delta1, delta2, delta3 = 1, 0.99, 0.85, 0.75  # difficulty reduction
deltas = np.array([delta0, delta1, delta2, delta3])
fs = np.array([f0, f1, f2, f3])  # pool fees
lams= lam / deltas # arrival rates of the pools
bs = b * (1 - fs) * deltas  # rewards after pool fees
X_pools = [wealth_process(lam, b, c, 0, 1 ) for lam, b in zip(lams, bs)]
# Compute the expected values and variance of the wealth process at time T=1 for each pool
V_pools = [X.V(x,q)[1] for X in X_pools]
V_pools
latex_table = pd.DataFrame({
    'pool': ["solo", "#1", "#2", "#3"],
    'share discovery rate': lams,
    'share reward': bs,
    'pool fee': fs,
    'difficulty reduction': deltas,
    'V': V_pools
}).sort_values(by='V', ascending=False).round(3).to_latex(index=False)
print(latex_table)
# pd.DataFrame({'pool': ["solo", "#1", "#2", "#3"], 'share discovery rate': lams, 'share reward': bs, 'pool fee': fs, 'difficulty reduction': deltas, 'V': V_pools}).sort_values(by='V', ascending=False)


\begin{tabular}{lrrrrr}
\toprule
pool & share discovery rate & share reward & pool fee & difficulty reduction & V \\
\midrule
#2 & 7.059000 & 2.630000 & 0.010000 & 0.850000 & 6.027000 \\
solo & 6.000000 & 3.125000 & 0.000000 & 1.000000 & 6.008000 \\
#1 & 6.061000 & 3.078000 & 0.005000 & 0.990000 & 5.935000 \\
#3 & 8.000000 & 2.109000 & 0.100000 & 0.750000 & 4.694000 \\
\bottomrule
\end{tabular}



## Best out of two pools

A simple univariate root finding algorithm is used to test all the combinations of two

In [11]:
prec = Decimal('0.001')  # precision for the position
res = {}
n_comb_max = 2
n = 3
for comb in itertools.combinations(range(n+1), n_comb_max):
    w = np.array([1 / 2, 1 / 2])
    X_mixed = wealth_process(lams[np.array(comb)], bs[np.array(comb)], c, 1, w)
    res_max_scalar = max_scalar(X_mixed, x, q)
    res_max_scalar['position'][res_max_scalar['position'] > 1 - prec] = 1
    res_max_scalar['position'][res_max_scalar['position'] < prec] = 0
    res[comb] = res_max_scalar
res
    

{(0, 1): {'position': array([1., 0.]), 'value': 6.008253184676769},
 (0, 2): {'position': array([0., 1.]), 'value': 6.027114331220455},
 (0, 3): {'position': array([1., 0.]), 'value': 6.0082451936751395},
 (1, 2): {'position': array([0., 1.]), 'value': 6.027113863079295},
 (1, 3): {'position': array([1., 0.]), 'value': 5.934919760761006},
 (2, 3): {'position': array([1., 0.]), 'value': 6.027105362401609}}